## Combine Models

According our analysis, we found out that Time Series Clustering Results with the best model.


Cluster 1: Arima Time Clustering

Cluster 2: GBoostRegression- sum unit

Cluster 3: XGBoostRegression w/ 2 temprature

Cluster 4: Windowing Time Series Clustering (Lag=20)- ANN

Cluster 5: XGBoostRegression w/ 2 temprature

Cluster 6: GBoostRegression- sum unit

Cluster 7: Windowing Individually Lag(30)- Linear Reg

Cluster 8: GBoostRegression w/ 2 temprature

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from datetime import date, timedelta
import datetime as dt

### Import data

In [2]:
data=pd.read_csv('integrated_data_Feb.csv')
cluster_time=pd.read_csv('time_cluster_bg.csv')

In [9]:
data_cluster=cluster_time[['key','clust_7']]
data=pd.merge(data,data_cluster,on='key',how='left')
data=data.fillna(8)

### Split train & test

In [11]:
train_df=data[data.month !=2] 
test_df=data[data.month ==2] 
test_df=test_df.reset_index(drop=True)

### Clustering

In [12]:
def cluster_2temp(train,test,label):
    df_train_data_x=train[train.clust_7 ==label]
    df_test_data_x=test[test.clust_7 ==label] 
    
    target_train_x = df_train_data_x[['units']]
    data_train_x = df_train_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit'], axis=1)
    target_test_x = df_test_data_x[['units']]
    data_test_x =df_test_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit'], axis=1)
    
    return target_train_x,data_train_x,target_test_x,data_test_x,df_test_data_x

In [13]:
X0=cluster_2temp(train_df,test_df,1)
X1=cluster_2temp(train_df,test_df,2)
X2=cluster_2temp(train_df,test_df,3)
X3=cluster_2temp(train_df,test_df,4)
X4=cluster_2temp(train_df,test_df,5)
X5=cluster_2temp(train_df,test_df,6)
X6=cluster_2temp(train_df,test_df,7)
X7=cluster_2temp(train_df,test_df,8)

In [14]:
def cluster_1temp(train,test,label):
    df_train_data_x=train[train.clust_7 ==label]
    df_test_data_x=test[test.clust_7 ==label] 
    
    target_train_x = df_train_data_x[['units']]
    data_train_x = df_train_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit','median_temp'], axis=1)
    target_test_x = df_test_data_x[['units']]
    data_test_x =df_test_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit','median_temp'], axis=1)
    
    return target_train_x,data_train_x,target_test_x,data_test_x,df_test_data_x
    

In [15]:
X1=cluster_1temp(train_df,test_df,2)

### Cluster 1 (Arima Time Clustering)

In [21]:
clust0=pd.read_excel('ARIMA_cluster_1_pre_units_Feb.xlsx')

In [27]:
test_clust0=X0[4][['key','day','units']]
test_clust0=test_clust0.reset_index(drop=True)
clust0_full=clust0.append([clust0]*(test_clust0.key.nunique()-1), ignore_index=True)
pre_cluster0=pd.merge(test_clust0,clust0_full, left_index=True, right_index=True)

pre_cluster1=pre_cluster0.rename(index=str, columns={'forecast of units': "pre_units"})

### Cluster 2 (GBoostRegression sum unit)

In [32]:
da_g=data[['key','month','units']]

oct_=da_g[da_g['month']==10]
nov_=da_g[da_g['month']==11]
dec_=da_g[da_g['month']==12]
Jan_=da_g[da_g['month']==1]

octt=oct_.groupby(['key', 'month'])['units'].sum()
novv=nov_.groupby(['key', 'month'])['units'].sum()
decc=dec_.groupby(['key', 'month'])['units'].sum()
jann=Jan_.groupby(['key', 'month'])['units'].sum()

octt=(pd.DataFrame(octt)).reset_index()
novv=(pd.DataFrame(novv)).reset_index()
decc=(pd.DataFrame(decc)).reset_index()
jann=(pd.DataFrame(decc)).reset_index()

octt=octt.rename(columns={'units':'prev_month_sales'})
octt[['month']]=11
novv=novv.rename(columns={'units':'prev_month_sales'})
novv[['month']]=12
decc=decc.rename(columns={'units':'prev_month_sales'})
decc[['month']]=1
jann=jann.rename(columns={'units':'prev_month_sales'})
jann[['month']]=2

In [33]:
nov_data=data[data.month ==11] 
dec_data=data[data.month ==12]
jan_data=data[data.month ==1]
feb_data=data[data.month ==2]

In [36]:
# trainning data
november=nov_data.merge(octt[['key','prev_month_sales']],on=['key'],how='left')
december=dec_data.merge(novv[['key','prev_month_sales']],on=['key'],how='left')
january=jan_data.merge(decc[['key','prev_month_sales']],on=['key'],how='left')

df_train_data_g=pd.concat([november,december,january])
#df_train_data_gg=pd.merge(df_train_data_g,cluster_time[['key','clust_7']],on='key',how='left')
#df_train_data_gg=df_train_data_gg.fillna(8) 

# testing data
df_test_data_g=feb_data.merge(jann[['key','prev_month_sales']],on=['key'],how='left')
#df_test_data_gg=pd.merge(df_test_data_g,cluster_time[['key','clust_7']],on='key',how='left')  
#df_test_data_gg=df_test_data_gg.fillna(8) 

# pick out cluster 2
X1_g=cluster_2temp(df_train_data_g,df_test_data_g,2)

In [43]:
from sklearn import ensemble
params2 = {'n_estimators': 1000, 'max_depth': 3, 'min_samples_split': 2,
              'learning_rate': 0.05, 'loss': 'ls'}
clf2 = ensemble.GradientBoostingRegressor(**params2)
clf2.fit(X1_g[1], X1_g[0]['units'].ravel()) 
prediction2= clf2.predict(X1_g[3])
    
df_prediction2=pd.DataFrame(prediction2, columns=['pre_units']).round(decimals=5)
df_prediction_need2=X1_g[4][['key','day','units']]
df_prediction_need2=df_prediction_need2.reset_index(drop=True)
pre_cluster2=df_prediction_need2.merge(df_prediction2, left_index=True, right_index=True)

### Cluster 3 (XGBoostRegression w/ 2 temprature)

In [50]:
from xgboost import XGBRegressor
def XGBoostingRegressor2(data_train_X, target_train_y, data_test_X, df_test_data_X):
    model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,n_estimators=1000,max_depth=6,min_child_weight=7,objective='reg:linear',subsample=0.8)
    model.fit(data_train_X, target_train_y['units'].ravel())
    # make predictions for test data
    prediction = model.predict(data_test_X)
    #predictions = [round(value) for value in y_pred]
    df_prediction=pd.DataFrame(prediction, columns=['pre_units']).round(decimals=5)
    df_prediction_need=df_test_data_X[['key','day','units']]
    df_prediction_need=df_prediction_need.reset_index(drop=True)
    df_prediction2=df_prediction_need.merge(df_prediction, left_index=True, right_index=True)
    return df_prediction2

pre_cluster3=XGBoostingRegressor2(X2[1], X2[0], X2[3],X2[4])

C:\Users\GONGNA\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Cluster 4 (Windowing Time Seies Clustering (Lag:20) ANN

In [51]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
        Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [52]:
d1 = date(2017, 10, 1)  # start date
d2 = date(2018, 1, 31)  # end date
d3=date(2018, 2, 28)

delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
train=train=train_df[['date','key','units']]


In [53]:
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})

In [54]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [55]:
sales_prev=train.loc[train['date']=='2018-01-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2018-01-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2018-01-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2018-01-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2018-01-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2018-01-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2018-01-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2018-01-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2018-01-23'].sort_values('date')[['key','units']] 
sales_prev9=train.loc[train['date']=='2018-01-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2018-01-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2018-01-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2018-01-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2018-01-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2018-01-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2018-01-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2018-01-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2018-01-14'].sort_values('date')[['key','units']]
sales_prev18=train.loc[train['date']=='2018-01-13'].sort_values('date')[['key','units']]
sales_prev19=train.loc[train['date']=='2018-01-12'].sort_values('date')[['key','units']]
sales_prev20=train.loc[train['date']=='2018-01-11'].sort_values('date')[['key','units']]
sales_prev21=train.loc[train['date']=='2018-01-10'].sort_values('date')[['key','units']]
sales_prev22=train.loc[train['date']=='2018-01-09'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-09'})


In [59]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [61]:
c_train=train.merge(data_cluster,on='key',how='inner') #get cluster for train
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner') #get clusers for test 

In [66]:
def days_20_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=20, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-20)', 'var1(t-19)', 'var1(t-18)', 'var1(t-17)',
       'var1(t-16)', 'var1(t-15)', 'var1(t-14)', 'var1(t-13)',
       'var1(t-12)', 'var1(t-11)', 'var1(t-10)', 'var1(t-9)', 'var1(t-8)',
       'var1(t-7)', 'var1(t-6)', 'var1(t-5)', 'var1(t-4)', 'var1(t-3)',
       'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,29)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=2)
        pred_date_before=predicted_date- timedelta(days=20)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [67]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor()
cluster4_frame=days_20_cluster(c_dataframe,c_train,4,mlp)

(1342, 3)
11


C:\Users\GONGNA\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
cluster4_reults=cluster4_frame.transpose()[0:28].values
cluster4_reults
test=test_df[['key','date','units']].merge(data_cluster, on='key',how='inner')
test_clust4=test[test['clust_7']==4]
i=0
n=test_clust4.key.nunique()
days=range(1,29)
day_all=[]
pre_unit=[]
while(i<n):
    k=0
    while(k<28):
        day_all.append(days[k])
        uni=cluster4_reults[k][0]
        pre_unit.append(uni)
        k=k+1
    i=i+1
test_clust4['day']=day_all
test_clust4['pre_units']=pre_unit
test_clust4=test_clust4.drop(['clust_7'],axis=1)

pre_cluster4=test_clust4[['key','day','units','pre_units']]

C:\Users\GONGNA\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GONGNA\Anaconda3\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Cluster 5 (XGBoostRegression w/ 2 temprature)

In [69]:
from xgboost import XGBRegressor
def XGBoostingRegressor2(data_train_X, target_train_y, data_test_X, df_test_data_X):
    model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,n_estimators=1000,max_depth=6,min_child_weight=7,objective='reg:linear',subsample=0.8)
    model.fit(data_train_X, target_train_y['units'].ravel())
    # make predictions for test data
    prediction = model.predict(data_test_X)
    #predictions = [round(value) for value in y_pred]
    df_prediction=pd.DataFrame(prediction, columns=['pre_units']).round(decimals=5)
    df_prediction_need=df_test_data_X[['key','day','units']]
    df_prediction_need=df_prediction_need.reset_index(drop=True)
    df_prediction2=df_prediction_need.merge(df_prediction, left_index=True, right_index=True)
    return df_prediction2

pre_cluster5=XGBoostingRegressor2(X4[1], X4[0], X4[3],X4[4])

### Cluster 6 (GBoost Sum Unit)

In [71]:
X5_g=cluster_2temp(df_train_data_g,df_test_data_g,6)

In [72]:
params5 = {'n_estimators': 1000, 'max_depth': 3, 'min_samples_split': 2,
              'learning_rate': 0.05, 'loss': 'ls'}
clf5 = ensemble.GradientBoostingRegressor(**params5)
clf5.fit(X5_g[1], X5_g[0]['units'].ravel()) 
prediction5= clf5.predict(X5_g[3])
    
df_prediction5=pd.DataFrame(prediction5, columns=['pre_units']).round(decimals=5)
df_prediction_need5=X5_g[4][['key','day','units']]
df_prediction_need5=df_prediction_need5.reset_index(drop=True)
pre_cluster6=df_prediction_need5.merge(df_prediction5, left_index=True, right_index=True)

### Cluster 7 (Windowing Individually Lag(30)-  Linear Reg)

In [73]:
allcomb=pd.concat([test_df,train_df])

In [74]:
allcomb=allcomb[allcomb['clust_7']==7]

In [77]:
year=2018
month=1
days=range(1, 29)
lag=2
col=post_day[1:] #column names for the df
col.append('key')
all_=allcomb.key.unique()
empty_matrix=np.zeros((all_.shape[0],len(col))) #column according to the 
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [81]:
sales_prev=allcomb.loc[allcomb['date']=='2018-01-31'].sort_values('date')[['key','units']]
sales_prev1=allcomb.loc[allcomb['date']=='2018-01-30'].sort_values('date')[['key','units']]
sales_prev2=allcomb.loc[allcomb['date']=='2018-01-29'].sort_values('date')[['key','units']]
sales_prev3=allcomb.loc[allcomb['date']=='2018-01-28'].sort_values('date')[['key','units']]
sales_prev4=allcomb.loc[allcomb['date']=='2018-01-27'].sort_values('date')[['key','units']]
sales_prev5=allcomb.loc[allcomb['date']=='2018-01-26'].sort_values('date')[['key','units']]
sales_prev6=allcomb.loc[allcomb['date']=='2018-01-25'].sort_values('date')[['key','units']]
sales_prev7=allcomb.loc[allcomb['date']=='2018-01-24'].sort_values('date')[['key','units']]
sales_prev8=allcomb.loc[allcomb['date']=='2018-01-23'].sort_values('date')[['key','units']] 
sales_prev9=allcomb.loc[allcomb['date']=='2018-01-22'].sort_values('date')[['key','units']]
sales_prev10=allcomb.loc[allcomb['date']=='2018-01-21'].sort_values('date')[['key','units']]
sales_prev11=allcomb.loc[allcomb['date']=='2018-01-20'].sort_values('date')[['key','units']]
sales_prev12=allcomb.loc[allcomb['date']=='2018-01-19'].sort_values('date')[['key','units']]
sales_prev13=allcomb.loc[allcomb['date']=='2018-01-18'].sort_values('date')[['key','units']]
sales_prev14=allcomb.loc[allcomb['date']=='2018-01-17'].sort_values('date')[['key','units']]
sales_prev15=allcomb.loc[allcomb['date']=='2018-01-16'].sort_values('date')[['key','units']]
sales_prev16=allcomb.loc[allcomb['date']=='2018-01-15'].sort_values('date')[['key','units']]
sales_prev17=allcomb.loc[allcomb['date']=='2018-01-14'].sort_values('date')[['key','units']]
sales_prev18=allcomb.loc[allcomb['date']=='2018-01-13'].sort_values('date')[['key','units']]
sales_prev19=allcomb.loc[allcomb['date']=='2018-01-12'].sort_values('date')[['key','units']]
sales_prev20=allcomb.loc[allcomb['date']=='2018-01-11'].sort_values('date')[['key','units']]
sales_prev21=allcomb.loc[allcomb['date']=='2018-01-10'].sort_values('date')[['key','units']]
sales_prev22=allcomb.loc[allcomb['date']=='2018-01-09'].sort_values('date')[['key','units']]
sales_prev23=allcomb.loc[allcomb['date']=='2018-01-08'].sort_values('date')[['key','units']]
sales_prev24=allcomb.loc[allcomb['date']=='2018-01-07'].sort_values('date')[['key','units']]
sales_prev25=allcomb.loc[allcomb['date']=='2018-01-06'].sort_values('date')[['key','units']]
sales_prev26=allcomb.loc[allcomb['date']=='2018-01-05'].sort_values('date')[['key','units']]
sales_prev27=allcomb.loc[allcomb['date']=='2018-01-04'].sort_values('date')[['key','units']]
sales_prev28=allcomb.loc[allcomb['date']=='2018-01-03'].sort_values('date')[['key','units']]
sales_prev29=allcomb.loc[allcomb['date']=='2018-01-02'].sort_values('date')[['key','units']]
sales_prev30=allcomb.loc[allcomb['date']=='2018-01-01'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-09'})
data_frame_2=data_frame_2.merge(sales_prev23, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-08'})
data_frame_2=data_frame_2.merge(sales_prev24, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-07'})
data_frame_2=data_frame_2.merge(sales_prev25, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-06'})
data_frame_2=data_frame_2.merge(sales_prev26, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-05'})
data_frame_2=data_frame_2.merge(sales_prev27, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-04'})
data_frame_2=data_frame_2.merge(sales_prev28, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-03'})
data_frame_2=data_frame_2.merge(sales_prev29, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-02'})
data_frame_2=data_frame_2.merge(sales_prev30, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2018-01-01'})
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [84]:
train_df_c7=train_df[train_df['clust_7']==7]

In [86]:
n=train_df_c7.key.nunique()
all_=train_df_c7.key.unique()
p=0
while(p<n): #looping over datasets
    #print(p)
    subset_tr=train_df_c7.loc[train_df_c7['key']==all_[p]].sort_values('date')
    df_model=series_to_supervised(subset_tr[['units']], n_in=30, n_out=1, dropnan=False)
    df_model=df_model.dropna(axis=0)
    days=range(1,29)
    regr = linear_model.LinearRegression()
    regr.fit( df_model[['var1(t-30)', 'var1(t-29)', 'var1(t-28)', 'var1(t-27)',
           'var1(t-26)', 'var1(t-25)', 'var1(t-24)', 'var1(t-23)',
           'var1(t-22)', 'var1(t-21)', 'var1(t-20)', 'var1(t-19)',
           'var1(t-18)', 'var1(t-17)', 'var1(t-16)', 'var1(t-15)',
           'var1(t-14)', 'var1(t-13)', 'var1(t-12)', 'var1(t-11)',
           'var1(t-10)', 'var1(t-9)', 'var1(t-8)', 'var1(t-7)', 'var1(t-6)',
           'var1(t-5)', 'var1(t-4)', 'var1(t-3)', 'var1(t-2)', 'var1(t-1)']],df_model[['var1(t)']])
    k=0
    day_n=0
    while(k<28): #looping over days
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=2)#this is the day i am trying to predict
        pred_date_before=predicted_date- timedelta(days=30)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=data_frame[bet_day].values[p].reshape(1, -1)
        y_pred = regr.predict(value_predict)
        data_frame[predicted_date].loc[p]=y_pred
        k=k+1
        day_n=day_n+1
    p=p+1

C:\Users\GONGNA\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [87]:
clust_7_frame=data_frame.set_index('key').transpose()[0:28]

#days=range(1,32)
a=clust_7_frame[['16818 L']]
a=a.rename(columns={'16818 L':'pre_units'}).reset_index()
a['day']=days
a['key']='16818 L'
b=clust_7_frame[['12985 L']]
b=b.rename(columns={'12985 L':'pre_units'}).reset_index()
b['day']=days
b['key']='12985 L'
c=clust_7_frame[['15845 L']]
c=c.rename(columns={'15845 L':'pre_units'}).reset_index()
c['day']=days
c['key']='15845 L'
d=clust_7_frame[['15845 M']]
d=d.rename(columns={'15845 M':'pre_units'}).reset_index()
d['day']=days
d['key']='15845 M'

all_=pd.concat([a,b,c,d])
all_=all_.rename(columns={'index':'date'})

In [88]:
cluster7_results=all_.merge(test_df[['key','date','units']],on=['key','date'],how='inner')
cluster7_results=cluster7_results.drop(['date'],axis=1)

i=0
n=cluster7_results.key.nunique()
days=range(1,29)
day_all=[]
while(i<n):
    k=0
    while(k<28):
        day_all.append(days[k])
        k=k+1
    i=i+1
    
cluster7_results['day']=day_all
pre_cluster7=cluster7_results[['key','day','units','pre_units']]

### Cluster 8 (GBoostRegression w/ 2 temprature)

In [90]:
from sklearn import ensemble

params7 = {'n_estimators': 1000, 'max_depth': 3, 'min_samples_split': 2,
              'learning_rate': 0.05, 'loss': 'ls'}
clf7 = ensemble.GradientBoostingRegressor(**params7)
clf7.fit(X7[1], X7[0]['units'].ravel()) 
prediction7= clf7.predict(X7[3])
    
df_prediction7=pd.DataFrame(prediction7, columns=['pre_units']).round(decimals=5)
df_prediction_need7=X7[4][['key','day','units']]
df_prediction_need7=df_prediction_need7.reset_index(drop=True)

pre_cluster8=df_prediction_need7.merge(df_prediction7, left_index=True, right_index=True)

### Integrate Prediction

In [92]:
pre_total = pd.concat([pre_cluster1,pre_cluster2,pre_cluster3,pre_cluster4,pre_cluster5,pre_cluster6,pre_cluster7,pre_cluster8])

In [98]:
import datetime

def predict_date(pre_total):
    date=[]
    i=0
    n=len(pre_total)
    while i<n:
        day_int=pre_total.iloc[i]['day']
        date_time=datetime.datetime(2018, 2, day_int, 0,0).date()
        date_string=str(date_time)
        date.append(date_string)
        i+=1
    pre_total['date']=date

    pre_total_pvt = pre_total.pivot_table('pre_units', ['key'], 'date')
    pre_total_pvt['key'] = pre_total_pvt.index
    pre_total_pvt=pre_total_pvt.reset_index(drop=True)

    return pre_total_pvt

In [99]:
pre_total_pvt    = predict_date(pre_total)

### Predict sold_out_date

In [101]:
items=pd.read_csv('items.csv',delimiter='|')
item_stock=items[['pid','size','stock']]
item_stock['key']=item_stock["pid"].astype(str) +" "+ item_stock["size"].astype(str)

C:\Users\GONGNA\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [114]:
from datetime import date, timedelta

d1 = date(2018, 2, 1)  
d2 = date(2018, 2, 28)  
delta = d2 - d1         
pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

sold_out_day=[]
key=[]
   
window_inner=item_stock.merge(pre_total_pvt,left_on='key', right_on='key', how='inner')
window_inner['soldOutDate']=0

all_=window_inner.key.unique()
n=len(all_) 
i=0

while(i<n): #dataset
    subset_w=pre_total_pvt.loc[pre_total_pvt['key']==all_[i]]
    stock=item_stock.loc[item_stock['key']==all_[i]]['stock'].values[0]
    n1=len(pre_day)
    k=0
    while(k<n1): #day calculation
        day=pre_day[k]
        that_day_sale=subset_w[day].values[0]
        stock=stock-that_day_sale
        if((stock==0) | (stock<0) ):
            window_inner.iloc[i,32]=day
            break
        if(k==(n1-1)): 
            window_inner.iloc[i,32]='2018-02-22'
        k=k+1
    i=i+1


In [119]:
window_inner['soldOutDate']=pd.to_datetime(window_inner['soldOutDate'])
SoldOutDay_predict=window_inner[['pid','size','soldOutDate']]

In [126]:
SoldOutDay_predict.head()

,pid,size,soldOutDate
0,10000,XL ( 158-170 ),2018-02-18
1,10001,L,2018-02-18
2,10003,3 (35-38 ),2018-02-11
3,10003,4 ( 39-42 ),2018-02-18
4,10003,5 ( 43-46 ),2018-02-18


In [127]:
SoldOutDay_predict.dtypes

pid                     int64
size                   object
soldOutDate    datetime64[ns]
dtype: object

In [131]:
SoldOutDay_predict.to_csv('SoldOutDay_predict.csv',sep='|',index=False)